<a href="https://colab.research.google.com/github/tejpat98/Textbook-Summarisation/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preprocessing textbook dataset:**
*   Download all textbooks (OP / CC licences)
*   Convert PDF into raw text data
*   Split textbooks into chapters
*   Put into a labeled dataset format


Notes:
https://stackoverflow.com/questions/60754884/python-ocr-pytesseract-for-pdf 


In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!pip install PyPDF2
!pip install pdf2image
!pip install pytesseract

In [ ]:
!nvidia-smi

Convert PDF into Images

In [ ]:
import os
from PIL import Image
from pdf2image import convert_from_path
import pytesseract

%cd '/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/1/'
path='/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/1/images/'
fileName = '1.pdf'
doc = convert_from_path(fileName, 300, output_folder=path, paths_only=True, fmt="jpeg")


Convert Images into Text using PyTesseract

In [ ]:
%cd '/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/1/images/'

ImageNames = !ls
print(len(ImageNames))
PrefList= ImageNames[0].strip().split('-')[:-1]
pref = '-'.join(PrefList)

for i in range(1,len(ImageNames)+1):
  with open('**output.txt', 'a') as f:
    txt = pytesseract.image_to_string(Image.open(str(pref)+'-'+str(i).zfill(3)+'.jpg'))
    f.write("*****Page: " + str(i) +"*****")
    f.write("\n")
    f.write(txt)